In [1]:
import numpy as np
import pandas as pd
from math import nan
import plotly.express as px
from plotly.io import to_html
from ridgeplot import ridgeplot
from collections import Counter
from random import shuffle, sample

pd.options.display.max_rows = 500

In [2]:
year_list = [2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

num_presentes = {'before': [7173563, 8722248, 7746427, 8627179, 6731278,
                            5513733, 5095171, 5783109, 3389832, 3476105],
                 'after': [5007934, 5947914, 5604905, 5818264, 4426692,
                           3893729, 3701909, 2588681, 2238107, 2344823]}

def stratify_sampling(x, number):
    """
    ARGUMENTS:
    x      = Python list
    number = integer indicating the number of samples you want
    """
    counts = Counter(x)
    unique = list(counts.keys())
    list_size = number/len(x)

    smp = []
    for i in range(len(unique)):
        batch = int(list_size*counts[unique[i]])
        if batch == 0: smp.append(unique[i])
        else:
            for j in range(batch): smp.append(unique[i])
    
    diff = number - len(smp)
    batch = sample(x, diff)
    for i in range(diff): smp.append(batch[i])
    
    shuffle(smp)
    
    return smp

In [3]:
df = pd.read_pickle('../../data/vis02_enem_data.pickle')
df

,TP_FAIXA_ETARIA,ANO,NOTA_MEDIA
0,6,2013,450
1,12,2013,380
2,3,2013,440
3,11,2013,510
4,2,2013,410
...,...,...,...
41572953,12,2022,480
41572954,11,2022,610
41572955,2,2022,540
41572956,11,2022,500


## Vis01 - Número de Presentes

In [4]:
array_dict = {'year': [], 'num_presence': [], 'Legenda': []}
for i in range(len(year_list)):
    array_dict['year'].append(year_list[i])
    array_dict['num_presence'].append(num_presentes['before'][i])
    array_dict['Legenda'].append('Total de Inscritos')

    array_dict['year'].append(year_list[i])
    array_dict['num_presence'].append(num_presentes['after'][i])
    array_dict['Legenda'].append('Presentes')

sub = pd.DataFrame(array_dict)

In [5]:
fig = px.line(sub, x='year', y='num_presence', color='Legenda', symbol="Legenda")

fig.update_layout(
    #title="Distribuição do Número de Presentes em Relação ao Ano",
    height=460,
    width=860,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    xaxis_gridcolor='rgba(0, 0, 0, 0.2)',
    yaxis_gridcolor='rgba(0, 0, 0, 0.2)',
    xaxis_title="Ano",
    yaxis_title="Número de Candidatos"
)

with open('../vis/vis01.html', 'w') as f:
    f.writelines(to_html(fig, include_plotlyjs='cnd', full_html=True))
print('Done!')

Done!


## Vis02 - NOTA_MEDIA

In [6]:
array_dict = {}
smp = 300000

for year in year_list:
    sub = df[df.ANO == year]
    array_dict[year] = sub['NOTA_MEDIA'].tolist()
    array_dict[year] = stratify_sampling(array_dict[year], smp)
    while len(array_dict[year]) < smp:
        array_dict[year].append(nan)

data = np.array([array_dict[2013], array_dict[2014], array_dict[2015],
                 array_dict[2016], array_dict[2017], array_dict[2018],
                 array_dict[2019], array_dict[2020], array_dict[2021],
                 array_dict[2022]])
graph = pd.DataFrame(columns=year_list, data=data.T)
graph

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,490,420,390,430,430,550,390,540,580,680
1,380,500,550,470,630,500,610,430,410,670
2,540,510,610,560,480,580,450,440,480,540
3,490,570,440,550,440,560,540,400,430,470
4,350,460,510,500,430,660,470,380,450,510
...,...,...,...,...,...,...,...,...,...,...
299995,540,450,580,520,450,550,540,470,570,520
299996,420,500,580,430,510,490,570,490,490,430
299997,540,410,580,440,610,480,550,510,660,400
299998,410,550,520,510,390,530,520,440,490,500


In [7]:
fig = ridgeplot(
    samples=graph.values.T,
    bandwidth=10,
    colorscale="viridis",
    colormode="index",
    coloralpha=0.6,
    labels=year_list,
    spacing=1
)

fig.update_layout(
    #title="Distribuição das Notas Médias em Relação ao Ano",
    height=700,
    width=860,
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    xaxis_gridcolor='rgba(0, 0, 0, 0.2)',
    yaxis_gridcolor='rgba(0, 0, 0, 0.2)',
    xaxis_title="Nota Média",
    yaxis_title="Ano"
)

with open('../vis/vis02.html', 'w') as f:
    f.writelines(to_html(fig, include_plotlyjs='cnd', full_html=True))
print('Done!')

Done!
